# LLM Pdf Reader
- [教學1](https://ithelp.ithome.com.tw/articles/10338349)
- [教學2](https://edge.aif.tw/express-langchain-chatpdf/)

- 參考以上兩個文章，並且稍微修改程式碼來免費讓LLM閱讀pdf的實作。
- 先安裝requirement.txt檔案內的套件。
- 我們會使用Groq提供的*llama3-70b-8192*模型，並且使用HuggingFace上的*sentence-transformers/all-MiniLM-L6-v2*模型來進行embedding，最後，使用*Chroma*來用vector db來儲存pdf text embeddings，作為LLM的回答參考依據。

In [ ]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [1]:
# 輸入groq的api，並且設為環境變數
api_path = "groq的api"
with open(api_path, "r") as f:
  api_key = f.read()
os.environ["GROQ_API_KEY"] = api_key

In [5]:
# 從HF上下載Embedding模型
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)

In [6]:
# 讀取檔案
file_path = "你的pdf檔案.pdf"
loader = file_path.endswith(".pdf") and PyPDFLoader(file_path) or TextLoader(file_path)

# 選擇 splitter 並將文字切分成多個 chunk
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = loader.load_and_split(splitter)

In [7]:
# 建立vector db，並設定retriever
'''
如果想要呼叫已經存好的vector db的話，可以改用下面這行:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
'''
persist_directory = '設定你要的db檔案名稱'
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)
retriever = vectordb.as_retriever()

In [8]:
# 使用chat
groq_llm = ChatGroq(temperature=0, model="llama3-70b-8192") # 可使用其他Groq支援的模型
qa = ConversationalRetrievalChain.from_llm(groq_llm, retriever)
chat_history = []

In [ ]:
# 跟LLM對話來快速抓取pdf的重點
while (query:= input('Enter query (q to quit) \n Q:')) != 'q:':
    result = qa({"question": query + ' (用英文回答)', "chat_history": chat_history}) # 可自行修改query
    print('A:', result['answer'])
    chat_history.append((query, result['answer']))

**心得**:
1. 目前是使用這個實作來幫助我快速熟悉文獻內容，回答的準確度不錯，LLM理解的內容也大部分都是正確的，所以是個能夠讓你快速理解pdf內容的好工具。
2. 唯一美中不足之處是有時LLM可能會裝作不知道內容，但換個方式問又可以得到跟你問題有關的回答，需要旁敲側擊個幾次。不過不是個太大的問題。